In [ ]:
import numpy as np
np.random.seed(0)
from global_vars import *
from commons import *
                                                                                                                                                                                                                                                                                                                                                            
import pickle as p
import glob 
import json
import subprocess
import getpass
import os
from nibabel.orientations import axcodes2ornt, ornt_transform, inv_ornt_aff, flip_axis
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

In [ ]:
def resize(img, shape=(256, 256, 400), is_label=False):
    if is_label:
        order = 0
        mode='nearest'
    else:
        order = 3
        mode='constant'
            
    print(img.shape)
    img = resample_from_to(img, [shape, img.affine], order=order, mode=mode, cval=0)
    return img

def crop(paths, shape, img=None):
    s1, e1, s2, e2, s3, e3 = shape
    for path in paths:
        img = nb.load(path) if img is None else img
        img_data= img.get_fdata()
        data = img_data[s1:e1, s2:e2, s3:e3]
        img = nb.Nifti1Image(data, img.affine, img.header)
        save_path = '/'.join(path.split('/')[:-1])
        vol_id = path.split('/')[-1].split('.')[0]
        save_volume(img, f'{save_path}_cropped/{vol_id}', np.int16)

In [ ]:
processed_dir = f'{DEFAULT_OUTPUT_PATH}/KORA/processed'
kora_labels = glob.glob(f'{processed_dir}/label/**')
kora_volumes = glob.glob(f'{processed_dir}/volume/**')
kora_volumes_f = glob.glob(f'{processed_dir}/volume_f/**')
kora_volumes_w = glob.glob(f'{processed_dir}/volume_w/**')
kora_volumes_in = glob.glob(f'{processed_dir}/volume_in/**')

processed_dir = f'{DEFAULT_OUTPUT_PATH}/NAKO/processed'
nako_labels = glob.glob(f'{processed_dir}/label/**')
nako_volumes = glob.glob(f'{processed_dir}/volume/**')
nako_volumes_f = glob.glob(f'{processed_dir}/volume_f/**')
nako_volumes_w = glob.glob(f'{processed_dir}/volume_w/**')
nako_volumes_in = glob.glob(f'{processed_dir}/volume_in/**')

processed_dir = f'{DEFAULT_OUTPUT_PATH}/UKB/processed'
ukb_labels = glob.glob(f'{processed_dir}/label/**')
ukb_volumes = glob.glob(f'{processed_dir}/volume/**')
ukb_volumes_f = glob.glob(f'{processed_dir}/volume_f/**')
ukb_volumes_w = glob.glob(f'{processed_dir}/volume_w/**')
ukb_volumes_in = glob.glob(f'{processed_dir}/volume_in/**')

all_labels =  kora_labels+nako_labels+ukb_labels
all_volumes =  kora_volumes + kora_volumes_f + kora_volumes_w +kora_volumes_in + nako_volumes + nako_volumes_f + nako_volumes_w +nako_volumes_in + ukb_volumes + ukb_volumes_f + ukb_volumes_w +ukb_volumes_in


In [ ]:
ukb_indices = [29, 221, 10, 186, 72, 184]
kora_indices = [32, 224, 2, 178, 69, 181]
nako_indices = [20, 212, 0, 176, 157, 269]

In [ ]:

for vol in all_volumes:
    print(vol)
    vol_id = vol.split('/')[-1].split('.')[0]
    ds = vol.split('/')[-4]
    save_path = '/'.join(vol.split('/')[:-1])
    print(ds, vol_id, save_path)
    vol_img = nb.load(vol)
    resized_img = resize(vol_img)
    if ds =='KORA':
        indices = kora_indices
    elif ds =='NAKO':
        indices = nako_indices
    elif ds =='UKB':
        indices = ukb_indices
    crop([vol], indices, resized_img)


In [ ]:
for vol in all_labels:
    vol_id = vol.split('/')[-1].split('.')[0]
    ds = vol.split('/')[-4]
    save_path = '/'.join(vol.split('/')[:-1])
    print(ds, vol_id, save_path)
    vol_img = nb.load(vol)
    resized_img = resize(vol_img, is_label=True)
    if ds =='KORA':
        indices = kora_indices
    elif ds =='NAKO':
        indices = nako_indices
    elif ds =='UKB':
        indices = ukb_indices
    crop([vol], indices, resized_img)

In [ ]:
# KORA = 158,100,91   ---  49, 206, 23, 122, 84, 176  --> 40, 210, 5, 145, 80, 180 == 170, 140, 100  == 176, 144, 112/96  (Multiples of 16)
#                                                       "40, 215, 5, 175, 70, 180" == "175, 170, 110" ==  '192 176 112'== 32, 224, 2, 178, 69, 181
# NAKO = 158,104,76    --- 44, 201, 40, 143, 169, 244  --> 41, 201, 23, 143, 169, 244 == 160, 120, 75 == 160, 128, 84 =AFTER FAT TALL MATCH= 
#                                                       31, 201, 23, 143, 160, 265 == "170, 120, 105" == '192 176 112' == 20, 212, 0, 176, 157, 269
# UKB= 163, 117, 97    --- 47, 209, 42, 158, 80, 176  --> 35, 215, 25, 170, 215, 315 == 180,145, 100 == 176/192, 144/160, 112 =AFTER FAT TALL[Missing] MATCH= 
#            (Evaluated on 6 merged UKB scans. BH+NOBH) 35, 215, 25, 170, 210, 315 == "180, 145, 105" == '192 176 112' == 29, 221, 10, 186, 72, 184  (for BH Merge only)
 
# KORA sample vol size = 250, 188, 370
# NAKO sample vol size = 226,184,316
# UKB sample vol size = 246, 137, 239
# sample target shape = 256, 256,400


In [ ]:
processed_dir = f'{DEFAULT_OUTPUT_PATH}/KORA/processed'
kora_labels_c = sorted(glob.glob(f'{processed_dir}/label_cropped/**'))
kora_volumes_c = sorted(glob.glob(f'{processed_dir}/volume_cropped/**'))
kora_volumes_c_f = sorted(glob.glob(f'{processed_dir}/volume_f_cropped/**'))
kora_volumes_c_w = sorted(glob.glob(f'{processed_dir}/volume_w_cropped/**'))
kora_volumes_c_in = sorted(glob.glob(f'{processed_dir}/volume_in_cropped/**'))

processed_dir = f'{DEFAULT_OUTPUT_PATH}/NAKO/processed'
nako_labels_c = sorted(glob.glob(f'{processed_dir}/label_cropped/**'))
nako_volumes_c = sorted(glob.glob(f'{processed_dir}/volume_cropped/**'))
nako_volumes_c_f = sorted(glob.glob(f'{processed_dir}/volume_f_cropped/**'))
nako_volumes_c_w = sorted(glob.glob(f'{processed_dir}/volume_w_cropped/**'))
nako_volumes_c_in = sorted(glob.glob(f'{processed_dir}/volume_in_cropped/**'))

processed_dir = f'{DEFAULT_OUTPUT_PATH}/UKB/processed'
ukb_labels_c = sorted(glob.glob(f'{processed_dir}/label_cropped/**'))
ukb_volumes_c = sorted(glob.glob(f'{processed_dir}/volume_cropped/**'))
ukb_volumes_c_f = sorted(glob.glob(f'{processed_dir}/volume_f_cropped/**'))
ukb_volumes_c_w = sorted(glob.glob(f'{processed_dir}/volume_w_cropped/**'))
ukb_volumes_c_in = sorted(glob.glob(f'{processed_dir}/volume_in_cropped/**'))


In [ ]:
labels_c = ukb_labels_c+nako_labels_c+kora_labels_c
volumes_c = ukb_volumes_c+ nako_volumes_c + kora_volumes_c
volumes_c_f = ukb_volumes_c_f+ nako_volumes_c_f + kora_volumes_c_f
volumes_c_w = ukb_volumes_c_w+ nako_volumes_c_w + kora_volumes_c_w
volumes_c_in = ukb_volumes_c_in+ nako_volumes_c_in + kora_volumes_c_in

In [ ]:
def visualize_and_save(volid, vol_root=f'{processed_dir}/label_cropped', label_root=f'{processed_dir}/volume_cropped', img_save_path = f'{processed_dir}/merged_imgs'):
    vol = nb.load(f'{vol_root}/{volid}.nii.gz')
    label = nb.load(f'{label_root}/{volid}.nii.gz') if label_root is not None else None

    im = vol.get_fdata()
    x = im.shape[2]//2
    masked = label.get_fdata() if label is not None else None
    plt.figure()
    plt.imshow(np.rot90(im[:,:,x]), 'gray', interpolation='none')
    if masked is not None:
        plt.imshow(np.rot90(masked[:,:,x]), 'jet', interpolation='none', alpha=0.5)
    plt.savefig(f'{img_save_path}/{volid}.png',  dpi=250, quality=95)
    plt.show()

In [ ]:
for p in volumes_c:
    volid = p.split('/')[-1].split('.')[0]
    vol_root= '/'.join(p.split('/')[:-2])+'/volume_cropped'
    label_root= '/'.join(p.split('/')[:-2])+'/label_cropped'
    img_save_path= '/'.join(p.split('/')[:-3])+'/merged_imgs_axial'
    print(volid, vol_root, label_root, img_save_path)
    create_if_not(img_save_path)
    visualize_and_save(volid, vol_root, label_root, img_save_path)

In [ ]:
phase_modes = {
    0: '',
    1: '',
    2: '_f',
    3: '_w',
    4: '_in'
}

references = {
    "KORA": {
        1: f'{DEFAULT_OUTPUT_PATH}/KORA/processed/volume_cropped/KORA2459745.nii.gz',
        2: f'{DEFAULT_OUTPUT_PATH}/KORA/processed/volume_f_cropped/KORA2459745.nii.gz',
        3: f'{DEFAULT_OUTPUT_PATH}/KORA/processed/volume_w_cropped/KORA2459745.nii.gz',
        4: f'{DEFAULT_OUTPUT_PATH}/KORA/processed/volume_in_cropped/KORA2459745.nii.gz',
    },
    "NAKO": {
        1: f'{DEFAULT_OUTPUT_PATH}/NAKO/processed/volume_cropped/100083.nii.gz',
        2: f'{DEFAULT_OUTPUT_PATH}/NAKO/processed/volume_f_cropped/100083.nii.gz',
        3: f'{DEFAULT_OUTPUT_PATH}/NAKO/processed/volume_w_cropped/100083.nii.gz',
        4: f'{DEFAULT_OUTPUT_PATH}/NAKO/processed/volume_in_cropped/100083.nii.gz',
    },
    "UKB": {
        1: f'{DEFAULT_OUTPUT_PATH}/UKB/processed/volume_cropped/1036900_20201_2_0.nii.gz',
        2: f'{DEFAULT_OUTPUT_PATH}/UKB/processed/volume_f_cropped/1036900_20201_2_0.nii.gz',
        3: f'{DEFAULT_OUTPUT_PATH}/UKB/processed/volume_w_cropped/1036900_20201_2_0.nii.gz',
        4: f'{DEFAULT_OUTPUT_PATH}/UKB/processed/volume_in_cropped/1036900_20201_2_0.nii.gz',
    }
}

kora_test_text = '/mnt/nas/Abhijit/Jyotirmay/abdominal_segmentation/lablmaps/KORA/testngb.txt'
nako_test_text = '/mnt/nas/Abhijit/Jyotirmay/abdominal_segmentation/lablmaps/NAKO/testngb.txt'
ukb_test_text = '/mnt/nas/Abhijit/Jyotirmay/abdominal_segmentation/lablmaps/UKB/testngb.txt'

all_val = ['KORA2460249', 'KORA2460734', '100006', '100008', '1004985_20201_2_0', '1004466_20201_2_0']

data_path = '/mnt/nas/Abhijit/Jyotirmay/abdominal_segmentation/dataset5'
all_test = []

all_test_paths = [kora_test_text, nako_test_text, ukb_test_text]
for volumes_txt_file in all_test_paths:
    with open(volumes_txt_file) as file_handle:
        volumes_to_use = file_handle.read().splitlines()
        print(volumes_txt_file, volumes_to_use)
        all_test.extend(volumes_to_use)

print(all_test)
exclude_list = ['volume', 'KORA', '.']
ngb_list = ['KORA2453290', '1108214_20201_2_0', '1584559_20201_2_0', '1002359_20201_2_0', '6025746_20201_2_0', '4627690_20201_2_0',
            '100183', '100161']

# for ps in zip(labels_c, volumes_c, volumes_c_f, volumes_c_w, volumes_c_in):
# for ps in zip(kora_labels_c, kora_volumes_c, kora_volumes_c_f, kora_volumes_c_w, kora_volumes_c_in):
for ps in zip(nako_labels_c, nako_volumes_c, nako_volumes_c_f, nako_volumes_c_w, nako_volumes_c_in):
# for ps in zip(ukb_labels_c, ukb_volumes_c, ukb_volumes_c_f, ukb_volumes_c_w, ukb_volumes_c_in):
    for i, p in enumerate(ps):
        print(i, p)
        ds = p.split('/')[-4]
        vol_id = p.split('/')[-1].split('.')[0]
        print('Processing: ', vol_id, ds)
        if vol_id in exclude_list:
            print('excluded:', vol_id)
            continue
        if vol_id in all_test:
            phase = 'test'
        elif vol_id in all_val:
            phase = 'val'
        else:
            phase = 'train'

        new_path = f'{data_path}/{ds}'
        all_path = f'{data_path}/ALL'
        if i == 0:
            label_img = nb.load(p)
            save_volume(label_img, f'{new_path}/{phase}/label/{vol_id}', np.int16)
            save_volume(label_img, f'{all_path}/{phase}/label/{vol_id}', np.int16)
        else:
            reference_path = None
            all_reference_path = references['NAKO'][i]
            reference_path = references[ds][i] if ds in references.keys() else None
            if reference_path == None:
                raise Exception('REFERENCE IS NONE')
            else:
                print(reference_path, all_reference_path)

            img = nb.load(p)

            save_volume(img, f'{new_path}/{phase}/volume{phase_modes[i]}/{vol_id}', np.int16)
            save_volume(img, f'{all_path}/{phase}/volume{phase_modes[i]}/{vol_id}', np.int16)

            hist_matched_img = hist_match(img, reference_path)
            save_volume(hist_matched_img, f'{new_path}/{phase}/volume_hist{phase_modes[i]}/{vol_id}', np.int16)
            hist_matched_img_all = hist_match(img, all_reference_path)
            save_volume(hist_matched_img_all, f'{all_path}/{phase}/volume_hist{phase_modes[i]}/{vol_id}', np.int16)

            intensity_matched_img = intensity_matching(img, reference_path)
            save_volume(intensity_matched_img, f'{new_path}/{phase}/volume_intensity{phase_modes[i]}/{vol_id}', np.int16)
            intensity_matched_img_all = intensity_matching(img, all_reference_path)
            save_volume(intensity_matched_img_all, f'{all_path}/{phase}/volume_intensity{phase_modes[i]}/{vol_id}', np.int16) 